In [1]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import pandas as pd
import numpy as np
import os

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Loading Wav2Vec pretrained model
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:733: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initiali

In [3]:
def load_audio(filename, sample_rate=16000):
    audio, rate = librosa.load(filename, sr = sample_rate)
    return audio

In [4]:
def transcribe_each(audio):
    # Taking an input value
    input_values = tokenizer(audio, return_tensors = "pt").input_values

    # Storing logits (non-normalized prediction values)
    logits = model(input_values).logits

    # Storing predicted ids
    prediction = torch.argmax(logits, dim = -1)

    # Passing the prediction to the tokenzer decode to get the transcription
    transcription = tokenizer.batch_decode(prediction)[0]
    
    return transcription

In [5]:
def transcribe(input_folder, output_excel):
    # List audio files in input folder
    audio_files = os.listdir(input_folder)

    # Extract actual word
    actual = input_folder[0:input_folder.rfind('_')]    ## ADJUST AS NEEDED 

    # Create empty datadrame
    df = pd.DataFrame(columns=['File', 'Actual', 'Predicted'])
    files = []
    actuals = []
    predicted = []

    for audio_file in audio_files:
        input_audio_file = os.path.join(input_folder, audio_file)

        # Check if it's a wav file (not a folder)
        if os.path.isfile(input_audio_file) and ".wav" in input_audio_file:

            # Transcribe .wav files 
            audio = load_audio(input_audio_file)
            transcription_each = transcribe_each(audio)
            files.append(input_audio_file)
            actuals.append(actual)
            predicted.append(transcription_each)

    # Write to excel file 
    df['File'] = files
    df['Actual'] = actuals
    df['Predicted'] = predicted
    df.to_excel(output_excel)

In [6]:
transcribe("andrew_wavs", "test.xlsx")